In [3]:
import sklearn
#from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
#from pmdarima.arima import auto_arima
#from keras.models import load_model
import joblib
import requests
import matplotlib.pyplot as plt
#from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# from keras.models import Sequential
# from keras.layers import Dense, LSTM
# from keras.models import load_model
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
# from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
# from statsmodels.tsa.seasonal import seasonal_decompose
# from keras.models import Sequential
# from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from datetime import datetime
from sklearn.linear_model import LinearRegression
import xgboost as xgb

from pyspark.sql.functions import year, month, dayofmonth, hour
from catboost import CatBoostRegressor
import pandas as pd

http://localhost:8070

In [2]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os
#https://repo1.maven.org/maven2/com/github/housepower/clickhouse-native-jdbc/2.7.1/clickhouse-native-jdbc-2.7.1.jar
packages = [
    "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3",
    "com.clickhouse:clickhouse-jdbc:0.6.0-patch4",
    "com.clickhouse:clickhouse-http-client:0.6.0-patch4",
    "org.apache.httpcomponents.client5:httpclient5:5.3.1",
    "com.github.housepower:clickhouse-native-jdbc:2.7.1"
]
ram = 12
cpu = 12*3
# Define the application name and setup session
appName = "Connect To ClickHouse via PySpark"
spark = (SparkSession.builder
         .appName(appName)
         # .config("spark.jars.packages", "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3,com.clickhouse:clickhouse-jdbc:0.6.0,com.clickhouse:clickhouse-http-client:0.6.0,org.apache.httpcomponents.client5:httpclient5:5.3.1,com.github.housepower:clickhouse-native-jdbc:2.7.1")
         .config("spark.jars.packages", ','.join(packages))
         .config("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
         # .config("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
         .config("spark.sql.catalog.clickhouse.host", "34.118.21.172") # from GCP VM instances or other server ip
         .config("spark.sql.catalog.clickhouse.protocol", "http")
         .config("spark.sql.catalog.clickhouse.http_port", "8123")
         .config("spark.sql.catalog.clickhouse.user", "default")
         .config("spark.sql.catalog.clickhouse.password", "1234")
         .config("spark.sql.catalog.clickhouse.database", "default")
         # .config("spark.spark.clickhouse.write.compression.codec", "lz4")
         # .config("spark.clickhouse.read.compression.codec", "lz4")
         # .config("spark.clickhouse.write.format", "arrow")
         #    .config("spark.clickhouse.write.distributed.convertLocal", "true") 
         #    .config("spark.clickhouse.write.repartitionNum", "1") 
         #.config("spark.clickhouse.write.maxRetry", "1000")
         #    .config("spark.clickhouse.write.repartitionStrictly", "true") 
         #    .config("spark.clickhouse.write.distributed.useClusterNodes", "false") 
        # .config("spark.clickhouse.write.batchSize", "10000000")
         # .config("spark.sql.catalog.clickhouse.socket_timeout", "600000000")
         # .config("spark.sql.catalog.clickhouse.connection_timeout", "600000000")
         # .config("spark.sql.catalog.clickhouse.query_timeout", "600000000")
         # .config("spark.clickhouse.options.socket_timeout", "600000000")
         # .config("spark.clickhouse.options.connection_timeout", "600000000")
         # .config("spark.clickhouse.options.query_timeout", "600000000")
         .config("spark.executor.memory", f"{ram}g")
         #.config("spark.executor.cores", "5")
         .config("spark.driver.maxResultSize", f"{ram}g")
         .config("spark.driver.memory", f"{ram}g")
         .config("spark.executor.memoryOverhead", f"{ram}g")
         #.config("spark.sql.debug.maxToStringFields", "100000")
         .getOrCreate()
         )
# spark.conf.set("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
# spark.conf.set("spark.sql.catalog.clickhouse.host", "127.0.0.1")
# spark.conf.set("spark.sql.catalog.clickhouse.protocol", "http")
# spark.conf.set("spark.sql.catalog.clickhouse.http_port", "8123")
# spark.conf.set("spark.sql.catalog.clickhouse.user", "default")
# spark.conf.set("spark.sql.catalog.clickhouse.password", "")
# spark.conf.set("spark.sql.catalog.clickhouse.database", "default")
spark.sql("use clickhouse")

In [3]:
df.show()

In [4]:
spark.sql('show databases').show()

In [3]:
#spark.sql('CREATE DATABASE first_database').show()

In [4]:
df = spark.read.csv('/projects/app/stations.csv',header=True,inferSchema=True)

In [2]:
df = spark.sql('''SELECT * FROM first_database.WeatherData''')

In [5]:

(
    df
    .write
    .mode("append")
    .format("jdbc")
    .option("url", "jdbc:clickhouse://34.118.0.142:9000/")
    .option("driver", "com.github.housepower.jdbc.ClickHouseDriver")
    .option("user", "default")
    .option("password", "1234")
    .option("dbtable", "first_database.WeatherDataSpark")
    .option("batchsize", "10000000")
    .option("numPartitions", cpu)
    .option("truncate", "true")
    .option("isolationLevel", "NONE")
    .save()
)


In [3]:
df.createOrReplaceTempView('df_sql')
spark.sql("""
INSERT INTO first_database.WeatherDataSpark
SELECT * FROM df_sql
""")

In [ ]:
spark.sql("""
SELECT * FROM default.input_test_table
""").show()

In [7]:
#df.createOrReplaceTempView('df_sql')
df = spark.sql("""
SELECT * FROM first_database.Weather_moscow_archive
""").show()

In [13]:
df = spark.sql("""
SELECT * FROM first_database.Weather_moscow_archive
""")

In [16]:
df.show()

In [12]:
from pyspark.sql import SparkSession

# Создание сессии Spark
# spark = SparkSession.builder \
#     .appName("Temperature Forecasting") \
#     .getOrCreate()

# # Загрузка данных из базы данных в Spark DataFrame
# jdbc_url = "jdbc:postgresql://your_database_host:your_database_port/your_database_name"
# table_name = "your_table_name"
# user = "your_username"
# password = "your_password"
# 
# df = spark.read.format("jdbc") \
#     .option("url", jdbc_url) \
#     .option("dbtable", table_name) \
#     .option("user", user) \
#     .option("password", password) \
#     .load()

# Предобработка данных
from pyspark.sql.functions import year, month, dayofmonth, hour

df = df.withColumn("Local time in Moscow", to_timestamp("Local time in Moscow", "dd.MM.yyyy HH:mm")) \
    .filter(df["Local time in Moscow"] < "2023-01-01")

df = df.withColumn("year", year("Local time in Moscow")) \
    .withColumn("month", month("Local time in Moscow")) \
    .withColumn("day", dayofmonth("Local time in Moscow")) \
    .withColumn("hour", hour("Local time in Moscow")) \
    .select("year", "month", "day", "hour", "T")

# Обучение модели с помощью CatBoost
from catboost import CatBoostRegressor

catboost_model = CatBoostRegressor()

catboost_model.fit(df.drop("T"), df["T"])

# Предсказание на тестовых данных
# Процесс предобработки и предсказания на тестовых данных аналогичен процессу обучения
# ...

# Закрытие сессии Spark
spark.stop()


downloading data and fitting model

In [55]:
clickhouse_df = spark.sql("""
SELECT * FROM first_database.Weather_moscow_archive
""")

In [56]:
clickhouse_df = clickhouse_df.withColumn("Local time in Moscow", F.to_timestamp(F.col("Local time in Moscow"), 'dd.MM.yyyy HH:mm'))
clickhouse_df.show()


In [59]:

clickhouse_df = spark.sql("""
SELECT * FROM first_database.Weather_moscow_archive
""")
# Обработка данных и подготовка их для обучения модели
clickhouse_df = clickhouse_df.withColumn("Local time in Moscow", F.to_timestamp(F.col("Local time in Moscow"), 'dd.MM.yyyy HH:mm'))

processed_df = clickhouse_df \
    .withColumn("year", F.year(F.col("Local time in Moscow"))) \
    .withColumn("month", F.month(F.col("Local time in Moscow"))) \
    .withColumn("day", F.dayofmonth(F.col("Local time in Moscow"))) \
    .withColumn("hour", F.hour(F.col("Local time in Moscow"))) \
    .select("year", "month", "day", "hour", "T","Local time in Moscow")

# Конвертация Spark DataFrame в Pandas DataFrame
pandas_df = processed_df.toPandas()

# Разделение данных на признаки и целевую переменную
feature_columns = ["year", "month", "day", "hour"]
label_column = "T"

# Создание и обучение модели CatBoost
catboost_model = CatBoostRegressor()
catboost_model.fit(pandas_df[feature_columns], pandas_df[label_column])
pandas_df

In [31]:
clickhouse_df.show()

In [62]:
new_data = ['2024-06-28 21:00:00']

def temp_pred_catb(data,model):
    # Преобразуйте новые данные в DataFrame и укажите правильное имя столбца
    data_df = pd.DataFrame(data, columns=['Local time in Moscow'])
    data_df['Local time in Moscow'] = pd.to_datetime(data_df['Local time in Moscow'], format='%Y-%m-%d %H:%M:%S')

    # Извлеките признаки из новых данных
    data_features = pd.DataFrame()
    data_features['year'] = data_df['Local time in Moscow'].dt.year
    data_features['month'] = data_df['Local time in Moscow'].dt.month
    data_features['day'] = data_df['Local time in Moscow'].dt.day
    data_features['hour'] = data_df['Local time in Moscow'].dt.hour
    print(data_features)
    # Сделайте предсказания
    predictions = model.predict(data_features)
    return predictions[0]

In [63]:
temp_pred_catb(new_data,catboost_model)

In [30]:
import joblib
  # Сохранение модели
joblib.dump(catboost_model, 'my_spark_catb_model.joblib')

In [31]:
# Загрузка модели
loaded_model = joblib.load('my_spark_catb_model.joblib')
# Загрузите новые данные и предобработайте их, извлекая признаки
# Пример: новые данные хранятся в переменной new_data
new_data = ['2024-07-30 21:00:00']



In [32]:
def temp_pred(data,model):
    # Преобразуйте новые данные в DataFrame и укажите правильное имя столбца
    data_df = pd.DataFrame(data, columns=['Local time in Moscow'])
    data_df['Local time in Moscow'] = pd.to_datetime(data_df['Local time in Moscow'], format='%Y-%m-%d %H:%M:%S')

    # Извлеките признаки из новых данных
    data_features = pd.DataFrame()
    data_features['year'] = data_df['Local time in Moscow'].dt.year
    data_features['month'] = data_df['Local time in Moscow'].dt.month
    data_features['day'] = data_df['Local time in Moscow'].dt.day
    data_features['hour'] = data_df['Local time in Moscow'].dt.hour

    # Сделайте предсказания
    predictions = model.predict(data_features)
    return predictions[0]

In [33]:
temp_pred(data=new_data, model=loaded_model)

##### testing join from 2 tables

In [35]:
#df.createOrReplaceTempView('df_sql')
df = spark.sql("""
SELECT localtime, Temperature_C FROM first_database.WeatherData 
ORDER BY localtime 
DESC LIMIT 1""")

# Получите временную метку из DataFrame
timestamp = df.collect()[0][0]
Temperature_C = df.collect()[0][1]
# Преобразуйте временную метку в строку в нужном формате
formatted_date = timestamp.strftime('%Y-%m-%d %H:%M:%S')

# Создайте словарь
X_dict = {'date': [formatted_date]}


In [5]:
predicted_data_response_xgb = requests.post('https://weatherimage-ma4ayonvha-lm.a.run.app/model/predict_xgb_t', json=X_dict) # local address

print(predicted_data_response_xgb)
print(predicted_data_response_xgb.text)

In [15]:
from io import StringIO

predicted_results_xgb = pd.read_json(StringIO(predicted_data_response_xgb.json()))

In [28]:
predicted_data_response_catb = requests.post('https://weatherimage-ma4ayonvha-lm.a.run.app/model/predict_catb_t', json=X_dict) # local address

print(predicted_data_response_catb)
print(predicted_data_response_catb.text)

In [41]:
predicted_results_catb = pd.read_json(StringIO(predicted_data_response_catb.json()))


In [39]:
round(predicted_results_xgb.iloc[0,0],2)

In [42]:
data_comparsion = pd.DataFrame()
data_comparsion['Localtime'] = [formatted_date]
data_comparsion['Temperature_C'] = [Temperature_C]
data_comparsion['Temperature_xgb_C'] = [round(predicted_results_xgb.iloc[0,0],2)]
data_comparsion['Temperature_catb_C'] = [round(predicted_results_catb.iloc[0,0],2)]
data_comparsion

In [43]:
data_comparsion.createOrReplaceTempView('data_comparsion')
spark.sql("""
INSERT INTO first_database.WeatherComparsion
SELECT * FROM data_comparsion
""")

In [44]:
data_comparsion.write.mode("overwrite").insertInto("first_database.WeatherComparsion")

In [3]:
from datetime import datetime
import requests
import pandas as pd
from io import StringIO
    
# Выполните запрос к базе данных для получения последней записи
df = spark.sql("""
SELECT localtime, Temperature_C FROM first_database.WeatherData 
ORDER BY localtime 
DESC LIMIT 1""")

# Получите временную метку и температуру из DataFrame
timestamp, temperature = df.collect()[0]

# Преобразуйте временную метку в нужный формат строки
formatted_date = timestamp.strftime('%Y-%m-%d %H:%M:%S')

# Создайте словарь для отправки запроса модели
X_dict = {'date': [formatted_date]}

# Отправьте запрос к модели для прогнозирования
predicted_data_response_xgb = requests.post('https://weatherimage-ma4ayonvha-lm.a.run.app/model/predict_xgb_t', json=X_dict)
predicted_data_response_catb = requests.post('https://weatherimage-ma4ayonvha-lm.a.run.app/model/predict_catb_t', json=X_dict)


predicted_results_xgb = pd.read_json(StringIO(predicted_data_response_xgb.json()))
predicted_results_catb = pd.read_json(StringIO(predicted_data_response_catb.json()))



# Создайте DataFrame для сравнения данных
data_comparison = pd.DataFrame({
    'Localtime': [formatted_date],
    'Temperature_C': [temperature],
    'Temperature_xgb_C': [round(predicted_results_xgb.iloc[0,0],2)],
    'Temperature_catb_C': [round(predicted_results_catb.iloc[0,0],2)]
})

data_comparison


#### Inserting archive update
 local kernel

In [25]:
import pandas as pd
from clickhouse_driver import Client

# Загрузка данных из CSV
csv_file_path = 'data/archive.csv'
df = pd.read_csv(csv_file_path)

# Преобразование столбцов с датами в нужный формат
df['Local time in Moscow'] = pd.to_datetime(df['Local time in Moscow'], format='%d.%m.%Y %H:%M')

# Удаление столбца ff10
df.drop(columns=['ff10','P','Pa', 'ff3', 'N', 'WW', 'W1', 'W2', 'Tn', 'Tx', 'Cl', 'H', 'Cm', 'Ch',
                 'Td', 'tR', 'Tg', 'RRR', 'E', "E'", 'sss'], inplace=True)

# Преобразование числовых столбцов в тип float
float_columns = ['T', 'Po',  'U', 'Ff',   ]
for col in float_columns:
    if col in df.columns:
        df[col] = df[col].astype(float)

# Преобразование строковых столбцов в тип string и обработка NaN значений
string_columns = ['DD', 'Nh',  'VV', ]
for col in string_columns:
    if col in df.columns:
        df[col] = df[col].astype(object).where(pd.notnull(df[col]), None)

# Подключение к ClickHouse через TCP (порт 9000)
client = Client(host='34.118.21.172', port=9000, user='default', password='1234', database='first_database')

# Преобразование данных DataFrame в список кортежей
data = [tuple(row) for row in df.itertuples(index=False, name=None)]

# Определение таблицы и колонок (если еще не создана таблица, создайте ее предварительно)
table_name = 'first_database.Weather_moscow_archive_updated'
columns = ', '.join([f'`{col}`' for col in df.columns])

# Вставка данных в таблицу ClickHouse
client.execute(f'INSERT INTO {table_name} ({columns}) VALUES', data, types_check=True)

print("Данные успешно записаны в ClickHouse")


In [ ]:
print()